In [61]:
import scrapy
import json
import requests
import requests_cache
import numpy as np
import pandas as pd
import seaborn as sns
import urlparse
import csv
import os
import os.path
from bs4 import BeautifulSoup
import sys
from lxml import html
import lxml.html as lx

requests_cache.install_cache('demo_cache')

In [62]:
def Box_search(year):
    
    base_url = 'http://www.boxofficemojo.com/yearly/chart/?yr='
    url = base_url + str(year)
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'lxml')
 
    res_2 = soup.find_all('td')
    names = []
    box = []
    studio = []
    for i in range(0,100):
        
        names.append(res_2[21+9*i].find_all('b')[0].text)
        box.append(res_2[23+9*i].find_all('b')[0].text)
        studio.append(res_2[22+9*i].find_all('a')[0].text)
        
        params = {'year':year, 'name':names, 'box':box,}
    
    return pd.DataFrame(params)

In [63]:
def Popular_Movies_search(year,pages,num):
    count = 0
    names = []
    
    for i in range(1,(pages+1)):
        url = "http://www.imdb.com/search/title?sort=moviemeter,asc&title_type=feature&year=" + str(year) + "," +str(year) + "&" + "page=" + str(i) + "&" + "ref_=adv_prv"        
        r = requests.get(url)
        soup = BeautifulSoup(r.content, "html.parser")
        search_results = soup.find_all("p", class_="sort-num_votes-visible")
        vote = int(search_results[0].find_all('span')[1].text.replace(',',''))
        soup = BeautifulSoup(r.content, "html.parser")
        search_results = soup.find_all("div", class_="lister-item-content")
        

        for result in search_results:
            try:
                vote = int(result.find_all('p')[3].find_all('span')[1].text.replace(',',''))
                if vote < 10000:
                    continue
            except:
                continue
            header = result.find("h3", class_="lister-item-header")
            title = header.a
            b = title.get_text().encode('utf-8')
            names.append(b)
            count = count + 1
            if count == num:
                return names
            
    return names

In [260]:
def Movie_info_search(term):
    print term
    url = 'http://www.omdbapi.com/'
    
    params = {'t':term}
    a_final = "&".join("{}={}".format(a,b) for a,b in params.items())
    url_final = url + '?' + a_final + '&tomatoes=true'
    response = requests.get(url_final)
    try:
        js = response.json()
        map(js.pop, ['tomatoFresh','tomatoConsensus','tomatoMeter','tomatoRating','tomatoRotten','tomatoURL','tomatoUserMeter','tomatoUserRating','tomatoUserReviews'])
    except:
        return {}
    return js

In [253]:
def Scoring_search(movie):
    imdb = Movie_info_search(movie).values()[-1]
    url = 'http://www.imdb.com/title/' + str(imdb) + '/ratings?ref_=tt_ov_rt'
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "html.parser")
    res_2 = soup.find_all('td')
    
    rates = []
    try:
        for i in xrange(14):
            number = str(res_2[37 + 3*i].text.replace(u'\xa0',u''))
            rate = str(res_2[38 + 3*i].find_all('img')[0].text.replace(u'\xa0',u''))
            rates.append([number + ',' + rate])
    except:
        return None
    params = {'males':rates[0], 'females':rates[1],'<18':rates[2], 'males<18':rates[3],'females<18':rates[4], '18-29':rates[5],'males 18-29':rates[6], 'females 18-29':rates[7],'30-44':rates[8],'males 30-44':rates[9], 'females 30-44':rates[10], '45+':rates[11],'males 45+':rates[12], 'females 45+':rates[13]}
    return params

In [71]:
Scoring_search('avatar')

{'18-29': ['370469,7.8'],
 '30-44': ['316818,7.9'],
 '45+': ['60290,7.9'],
 '<18': ['3290,7.7'],
 'females': ['143599,7.8'],
 'females 18-29': ['78922,7.7'],
 'females 30-44': ['47441,7.9'],
 'females 45+': ['9709,8.0'],
 'females<18': ['882,7.9'],
 'males': ['633253,7.9'],
 'males 18-29': ['288476,7.8'],
 'males 30-44': ['266060,7.9'],
 'males 45+': ['49694,7.9'],
 'males<18': ['2385,7.6']}

In [132]:
def Comment_search(movie,gender):
    comments = []
    imdb = imdb = Movie_info_search(movie).values()[-1]
    url = 'http://www.imdb.com/title/'+str(imdb)+'/reviews?filter=custom&spoiler=&vote_min=&vote_max=&gender='+gender+'&age_min=&age_max=&country=&order=score&custom=1'
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "html.parser")
    div = soup.find('div', attrs = {'id' : 'tn15content'}).find_all('p')
    for contents in div:
        
        comments.append(contents.text.replace('\n',' '))
    
    return comments
    

In [172]:
#div = soup.find('div', attrs = {'id' : 'tn15content'}).find_all('p')
#Comment_search('la la land','m')

In [151]:
film_2002 = Popular_Movies_search(2002,10,100)

#file_2006_dat = pd.DataFrame.from_dict(Movie_info_search(i) for i in film_2006) 

In [261]:
films_name = []
films_data = []
for year in xrange(0,15):
    films_name.append(Popular_Movies_search(year+2002,10,100))
    films_data.append(pd.DataFrame.from_dict(Movie_info_search(i) for i in films_name[year]))
    


The Ring
Catch Me If You Can
Gangs of New York
Road to Perdition
Spider-Man
City of God
Harry Potter and the Chamber of Secrets
The Lord of the Rings: The Two Towers
Resident Evil
Chicago
The Pianist
Minority Report
28 Days Later...
Men in Black II
A Walk to Remember
8 Mile
Irreversible
Signs
The Count of Monte Cristo
Secretary
xXx
Two Weeks Notice
Star Wars: Episode II - Attack of the Clones
The Bourne Identity
Panic Room
Scooby-Doo
We Were Soldiers
Insomnia
The Hot Chick
Red Dragon
Die Another Day
Sweet Home Alabama
Bend It Like Beckham
Unfaithful
Ying xiong
Lilo & Stitch
The Hours
Equilibrium
Austin Powers in Goldmember
Reign of Fire
The Transporter
About a Boy
Blade II
Blue Crush
Adaptation.
Ice Age
My Big Fat Greek Wedding
Treasure Planet
Ken Park
John Q
Crossroads
Star Trek: Nemesis
The Time Machine
Solaris
Punch-Drunk Love
Frida
Maid in Manhattan
Phone Booth
Spirit: Stallion of the Cimarron
Cabin Fever
The Scorpion King
Ghost Ship
Van Wilder
Infernal Affairs
Queen of the Damned


In [262]:
films_data[0]

,Actors,Awards,BoxOffice,Country,DVD,Director,Genre,Language,Metascore,Plot,...,Title,Type,Website,Writer,Year,imdbID,imdbRating,imdbVotes,tomatoImage,tomatoReviews
0,"Naomi Watts, Martin Henderson, David Dorfman, ...",14 wins & 11 nominations.,"$128,579,698.00","USA, Japan",04 Mar 2003,Gore Verbinski,"Horror, Mystery",English,57,A journalist must investigate a mysterious vid...,...,The Ring,movie,http://www.ring-themovie.com,"Ehren Kruger (screenplay), Kôji Suzuki (novel)",2002,tt0298130,7.1,"262,733",fresh,202
1,"Leonardo DiCaprio, Tom Hanks, Christopher Walk...",Nominated for 2 Oscars. Another 12 wins & 39 n...,"$164,400,000.00","USA, Canada",06 May 2003,Steven Spielberg,"Biography, Crime, Drama","English, French",76,"The true story of Frank Abagnale Jr. who, befo...",...,Catch Me If You Can,movie,http://www.dreamworks.com/catchthem/jump2.html,"Jeff Nathanson (screenplay), Frank Abagnale Jr...",2002,tt0264464,8.0,"564,278",certified,195
2,"Leonardo DiCaprio, Daniel Day-Lewis, Cameron D...",Nominated for 10 Oscars. Another 49 wins & 118...,"$77,605,296.00","USA, Italy",01 Jul 2003,Martin Scorsese,"Crime, Drama","English, Irish, Chinese, Latin",72,"In 1863, Amsterdam Vallon returns to the Five ...",...,Gangs of New York,movie,http://www.gangsofnewyork.com/,"Jay Cocks (story), Jay Cocks (screenplay), Ste...",2002,tt0217505,7.5,"327,657",certified,202
3,"Tyler Hoechlin, Rob Maxey, Liam Aiken, Jennife...",Won 1 Oscar. Another 21 wins & 76 nominations.,"$104,054,514.00",USA,25 Feb 2003,Sam Mendes,"Crime, Drama, Thriller",English,72,Bonds of loyalty are put to the test when a hi...,...,Road to Perdition,movie,http://www.roadtoperdition.com/,"Max Allan Collins (graphic novel), Richard Pie...",2002,tt0257044,7.7,"206,372",certified,211
4,"Tobey Maguire, Willem Dafoe, Kirsten Dunst, Ja...",Nominated for 2 Oscars. Another 14 wins & 56 n...,"$403,706,375.00",USA,01 Nov 2002,Sam Raimi,"Action, Adventure",English,73,"When bitten by a genetically modified spider, ...",...,Spider-Man,movie,http://spiderman.sonypictures.com/,"Stan Lee (Marvel comic book), Steve Ditko (Mar...",2002,tt0145487,7.3,"562,630",certified,236
5,"Alexandre Rodrigues, Leandro Firmino, Phellipe...",Nominated for 4 Oscars. Another 65 wins & 37 n...,N/A,"Brazil, France",08 Jun 2004,"Fernando Meirelles, Kátia Lund","Crime, Drama",Portuguese,79,Two boys growing up in a violent neighborhood ...,...,City of God,movie,http://www.miramax.com/movie/city-of-god,"Paulo Lins (novel), Bráulio Mantovani (screenp...",2002,tt0317248,8.7,"557,588",certified,156
6,"Daniel Radcliffe, Rupert Grint, Emma Watson, R...",Nominated for 3 BAFTA Film Awards. Another 11 ...,"$261,835,892.00","UK, USA, Germany",11 Apr 2003,Chris Columbus,"Adventure, Family, Fantasy",English,63,Harry ignores warnings not to return to Hogwar...,...,Harry Potter and the Chamber of Secrets,movie,http://harrypotter.warnerbros.com/,"J.K. Rowling (novel), Steve Kloves (screenplay)",2002,tt0295297,7.4,"410,363",certified,230
7,"Bruce Allpress, Sean Astin, John Bach, Sala Baker",Won 2 Oscars. Another 114 wins & 127 nominations.,"$339,700,000.00","USA, New Zealand",26 Aug 2003,Peter Jackson,"Adventure, Drama, Fantasy","English, Sindarin, Old English",88,While Frodo and Sam edge closer to Mordor with...,...,The Lord of the Rings: The Two Towers,movie,http://www.lordoftherings.net/,"J.R.R. Tolkien (novel), Fran Walsh (screenplay...",2002,tt0167261,8.7,"1,148,935",certified,248
8,"Ryan McCluskey, Oscar Pearce, Indra Ové, Anna ...",6 nominations.,"$39,532,308.00","UK, Germany, France, USA",30 Jul 2002,Paul W.S. Anderson,"Action, Horror, Sci-Fi",English,33,"A special military unit fights a powerful, out...",...,Resident Evil,movie,http://www.spe.sony.com/movies/residentevil,Paul W.S. Anderson,2002,tt0120804,6.7,"205,614",rotten,124
9,"Taye Diggs, Cliff Saunders, Catherine Zeta-Jon...",Won 6 Oscars. Another 49 wins & 121 nominations.,N/A,"USA, Germany, Canada",19 Aug 2003,Rob Marshall,"Comedy, Crime, Musical","English, Hungarian

In [264]:
#
year = 2002
for i in films_data:
    i.to_csv(str(year)+'.csv')
    year = year + 1

UnicodeEncodeError: 'ascii' codec can't encode character u'\xf4' in position 28: ordinal not in range(128)